In [1]:
import requests
import os
from PIL import Image
import base64
import numpy as np
import io
import glob
import json
from io import BytesIO
import asyncio
import aiohttp
import time
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def imagePathToBase64(imagePath):

    with open(imagePath, "rb") as imageFile:
        encodedString = base64.b64encode(imageFile.read())

    return encodedString.decode('utf-8')


In [3]:
def loadImageFile(imageFile):

        if imageFile.startswith('http') or imageFile.startswith('https'):
            response = requests.get(imageFile)
            image = Image.open(io.BytesIO(response.content)).convert('RGB')
        else:
            image = Image.open(imageFile).convert('RGB')
        return image


In [4]:
def imageResize(image, size):

    width = image.size[0]
    height = image.size[1]
    
    sizeW = int(size * width / height)
    sizeH = size

    return image.resize((sizeW, sizeH), Image.ANTIALIAS)


In [5]:
def arrayChunk(l, n): 
      
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [6]:
async def fetchUrl(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.post(url, data=data) as response:
            return await response.text()

In [7]:
async def fetchUrlList(urlList, dataList):
    
    tasks = []
    for i in range(0, len(urlList)):
        tasks.append(fetchUrl(urlList[i], dataList[i]))
    
    results = await asyncio.gather(*tasks)
    
    return results

In [8]:
async def filePathToFaceAge(apiEndpoint, fileList, minImageHeight, maxiImageHeight):

    filePathToPredictedAge = {}
    
    for i in range(0, len(fileList)):

        if i%10 == 0:
            print(i)
        
        urlList = []
        dataList = []

        for j in range(0, len(fileList[i])):

            imageToPredict = fileList[i][j]
            imageBin = loadImageFile(imageToPredict)
            
            # image height is to small, ignore it
            if imageBin.size[1] < minImageHeight:
                continue

            # image height is to big, resize it
            if imageBin.size[1] > maxiImageHeight:
                imageBinResize = imageResize(imageBin, maxiImageHeight)
                buffered = BytesIO()
                imageBinResize.save(buffered, format="JPEG")
                base64Image = base64.b64encode(buffered.getvalue()).decode('utf-8')
            else:
                base64Image = imagePathToBase64(imageToPredict)

            base64Image = 'data:image/jpeg;base64,' + base64Image

            urlList.append(apiEndpoint)
            dataList.append({
                'base64Image': base64Image,
                'imageToPredictPath': imageToPredict,
            })

        # call the face api
        results = await fetchUrlList(urlList, dataList)

        for k in range(0, len(results)):

            responseData = json.loads(results[k])

            # api responded with error
            if 'error' in responseData:
                continue

            # more than one face is detected
            if len(responseData['content']['ageList']) > 1:
                continue
                
            imageToPredictPath = responseData['content']['imageToPredictPath']
            predictedAge = responseData['content']['ageList'][0]
            
            filePathToPredictedAge[imageToPredictPath] = predictedAge

        time.sleep(0.2)
        
    return filePathToPredictedAge


In [9]:
def getCsvData(csvFilePath):
    
    with open(csvFilePath, newline='') as csvFile:
        reader = csv.reader(csvFile, delimiter=',')
        result = list(reader)

    return result

In [10]:
def filePathToDatasetData(fileList):
    
    filePathToData = {}
    
    for i in range(0, len(fileList)):
        
        imageToPredict = fileList[i]
        
        fileName = os.path.basename(imageToPredict)
        imageParts = imageToPredict.split('/')
    
        ageParts = imageParts[5].split('_')
        
        realAge = [int(ageParts[1]), int(ageParts[2])]
        gender = ''
        if imageParts[4] == 'female':
            gender = 'f'
        
        if imageParts[4] == 'male':
            gender = 'm'
        
        filePathToData[imageToPredict] = {
            'realAge': realAge,
            'gender': gender
        }
        
    
    return filePathToData
    

In [11]:
def createCsvContent(filePathToDatasetDataList, filePathToFaceAgeList):
    
    csvContent = [
        ['Real age', 'Gender', 'Predicted age', 'Absolute age difference']
    ]
    
    for filePath in filePathToFaceAgeList:
        
        realAge = str(filePathToDatasetDataList[filePath]['realAge'][0]) + '-' + str(filePathToDatasetDataList[filePath]['realAge'][1])
        gender = filePathToDatasetDataList[filePath]['gender']
        predictedAge = filePathToFaceAgeList[filePath]
        
        if predictedAge < int(filePathToDatasetDataList[filePath]['realAge'][0]):
            absoluteAgeDifference = abs(int(filePathToDatasetDataList[filePath]['realAge'][0]) - int(predictedAge))
        elif predictedAge > int(filePathToDatasetDataList[filePath]['realAge'][1]):
            absoluteAgeDifference = abs(int(filePathToDatasetDataList[filePath]['realAge'][1]) - int(predictedAge))
        else:
            absoluteAgeDifference = 0
        
        if int(filePathToDatasetDataList[filePath]['realAge'][0]) < 6:
            continue
            
        if int(filePathToDatasetDataList[filePath]['realAge'][1]) > 70:
            continue
        
        csvContent.append(
            [realAge, gender, predictedAge, absoluteAgeDifference]
        )
        
    return csvContent
    

In [12]:
def writeCsvFile(filePath, fileContent):
    
    with open(filePath, 'w', newline='') as csvfile:
        csvFile = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        for i in range(0, len(fileContent)):
            
            csvFile.writerow(fileContent[i])
            
    print('Csv write done!')
    

In [13]:
ARRAY_CHUNK_SIZE = 40
API_ENDPOINT = 'http://localhost:3200/predictAgeSimple'
IMAGE_SIZE_MIN = 100
IMAGE_SIZE_MAX = 600
DATASET_NAME = 'agfw'


In [14]:
fileList = glob.glob('../' + DATASET_NAME + '/cropped/*/*/*/*')
np.random.shuffle(fileList)
print('Total files:', len(fileList))

Total files: 36299


In [15]:
fileListChunked = list(arrayChunk(fileList, ARRAY_CHUNK_SIZE))
print('Total file chunks:', len(fileListChunked))

Total file chunks: 908


In [16]:
filePathToDatasetDataList = filePathToDatasetData(fileList)
print('Total data points:', len(filePathToDatasetDataList))
print('Data point sample:', filePathToDatasetDataList[fileList[0]])

Total data points: 36299
Data point sample: {'realAge': [45, 49], 'gender': 'f'}


In [17]:
filePathToFaceAgeList = await filePathToFaceAge(API_ENDPOINT, fileListChunked, IMAGE_SIZE_MIN, IMAGE_SIZE_MAX)
print('Total predictions:', len(filePathToFaceAgeList))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
Total predictions: 27499


In [18]:
csvContent = createCsvContent(filePathToDatasetDataList, filePathToFaceAgeList)
print('Total csv data entries:', len(csvContent) - 1)
print('Csv entry sample:', csvContent[1])

Total csv data entries: 26347
Csv entry sample: ['45-49', 'f', 56, 7]


In [19]:
writeCsvFile('results/' + DATASET_NAME + '-' + str(IMAGE_SIZE_MIN) + '.csv', csvContent)

Csv write done!
